In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [ ]:
events_df.columns

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad de eventos totales por usuario

In [44]:
df = events_df.groupby('person').event.count()

In [45]:
features_df = features_df.join(df, on='person')

In [46]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad de cada evento por usuario

In [40]:
df = events_df.groupby('person').event.value_counts()

In [41]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [42]:
df = df.drop(columns=['lead'])

In [43]:
df['ach*bl'] = (df['ad campaign hit'])*(df['brand listing'])
df['ach*gl'] = (df['ad campaign hit'])*(df['generic listing'])
df['ach*seh'] = (df['ad campaign hit'])*(df['search engine hit'])
df['ach*sp'] = (df['ad campaign hit'])*(df['searched products'])
df['ach*vp'] = (df['ad campaign hit'])*(df['viewed product'])
df['ach*vs'] = (df['ad campaign hit'])*(df['visited site'])

In [44]:
features_df = features_df.join(df, on='person')

In [45]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## cantidad de returnings

In [ ]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [ ]:
features_df = features_df.join(df, on='person')

## Quincenas

In [46]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [47]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [48]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [49]:
features_df = features_df.join(df, on='person')

## Dia mas frecuente

In [8]:
# Los mas frecuentes son martes, miercoles y jueves
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [ ]:
events_df.head(3)

In [ ]:
df = events_df.groupby('person').weekday.value_counts()

In [ ]:
df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday']).head()

In [ ]:
df = df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday', 'Thursday'])
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.head()

## Support para dia

In [9]:
events_df.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,mes,dia,weekday
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,Friday
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,Friday
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,Friday


In [9]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct = ct.astype('bool')

In [12]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.4, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.414098,(Thursday),1
1,0.404466,(Tuesday),1
2,0.416982,(Wednesday),1


In [13]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [14]:
result = columna_por_persona(ct)

In [15]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support   

In [16]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value 

In [17]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38829

In [18]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas 

In [19]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38829

In [20]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [21]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,weekday_support
0,4886f805,0.0
1,ad93850f,0.0
2,0297fc1e,0.0


## Es de Brasil o no (Agregarlo con los ev baja)

In [ ]:
def completar_dia_brasil(columna_country, dia, mes):
    columna_dia = list();
    columna_mes = list();
    for value in columna_country:
        if value=='Brazil':
            columna_dia.append(dia)
            columna_mes.append(mes)
        else:
            columna_dia.append(np.nan)
            columna_mes.append(np.nan)
    
    return columna_dia, columna_mes                

In [ ]:
columna_dia, columna_mes = completar_dia_brasil(events_df.country, 12, 5)

In [ ]:
events_df['dia_madre'] = columna_dia

In [ ]:
events_df['mes_madre'] = columna_mes

In [ ]:
def completar_ingreso_dia_brasil(columna_country, col_dia, col_mes):
    columna_ingreso = list();
    for i in range(0, len(columna_country)):
        if columna_country[i]=='Brazil':
            if (col_dia[i]>7 and col_dia[i]<13) and col_mes[i]==5:
                    columna_ingreso.append(True)
            else:
                columna_ingreso.append(False)
        else:
            columna_ingreso.append(False)
    
    return columna_ingreso                

In [ ]:
columna_ingreso = completar_ingreso_dia_brasil(events_df.country, events_df.dia, events_df.mes)

In [ ]:
len(columna_ingreso)

In [ ]:
events_df.shape

In [ ]:
events_df.country.shape

In [ ]:
events_df['ingreso'] = columna_ingreso

In [ ]:
df = events_df.groupby('person').ingreso.value_counts().unstack()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.drop(columns=[False])
features_df.columns
features_df.head()

## Tiempo total

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [ ]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_total'})

In [ ]:
features_df[['person', 'tiempo_total']].head()

In [ ]:
features_df.tiempo_total = pd.to_numeric(features_df.tiempo_total, downcast='float')
features_df.tiempo_total.head()

## Support para color (Al agregarlo baja)

In [ ]:
ct = pd.crosstab(events_df.person, events_df.color)
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [ ]:
result = columna_por_persona(ct)

In [ ]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support    

In [ ]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value 

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas 

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'color_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Labels y separacion de df

In [22]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [50]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 29)

In [51]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 28)

In [52]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [53]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [54]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [28]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [29]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [30]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3, 0.5],
        'colsample_bylevel': [0.7, 0.8, 0.9],
        'max_depth': [3, 4, 5, 7, 8, 9, 11, 13, 15],
        'base_score': [0.1, 0.2, 0.3],
        'n_estimators': [35, 40, 50, 60, 70, 80, 90]
        }

In [31]:
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [32]:
xg_gs = xgb.XGBRegressor(objective ='binary:logistic', booster='dart', 
                        learning_rate = 0.2,
                        alpha = 20, eval_metric='auc')

In [55]:
folds = 10
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_gs, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, 
                                   cv=skf.split(train_df.drop(columns=['person']),labels), verbose=3, random_state=1001,return_train_score=True)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_df.drop(columns=['person']), labels)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   20.3s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:  1.0min finished



 Time taken: 0 hours 1 minutes and 4.71 seconds.


In [56]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([2.23229496, 3.1355608 , 5.00180414, 2.03392346, 2.00962379,
       8.0281297 , 1.16872776]), 'std_fit_time': array([0.12681484, 0.07961579, 0.09316182, 0.03375388, 0.05735438,
       0.19884984, 0.0344566 ]), 'mean_score_time': array([0.01608121, 0.01163032, 0.02280328, 0.01534622, 0.01152992,
       0.02453156, 0.01216145]), 'std_score_time': array([0.00031942, 0.00037077, 0.00096056, 0.00021081, 0.00039369,
       0.00312664, 0.00030669]), 'param_n_estimators': masked_array(data=[50, 80, 70, 50, 90, 80, 60],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 10, 1, 5, 10, 5, 10],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[8, 7, 9, 7, 4, 13, 5],
             mask=[False, False, False, False, False, False, False],
      

In [57]:
preds_grid_search = random_search.predict(X_test)

In [58]:
roc_auc_score(y_test, preds_grid_search)

0.8684722748945742

In [59]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
preds = xg_reg.predict(X_test)

In [61]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,preds)

0.8345496594935027

In [39]:
X_train = X_train.fillna(0)

In [40]:
#Select kbest
from sklearn.feature_selection import SelectKBest, roc_auc
X_new = SelectKBest(auc, k=20).fit_transform(X_train, y_train)

ImportError: cannot import name 'roc_auc'

## Prediccion

In [62]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [63]:
preds_posta

array([0.00709401, 0.03323603, 0.01390382, ..., 0.11214011, 0.09837081,
       0.09837081], dtype=float32)

In [ ]:
preds_posta_sin_g_s = xg_reg.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_posta_sin_g_s

## Genero csv a publicar

In [64]:
to_publish = pd.DataFrame()

In [65]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [66]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [67]:
to_publish.head()

,person,label
0,4886f805,0.007094
1,0297fc1e,0.033236
2,2d681dd8,0.013904
3,cccea85e,0.028492
4,4c8a8b93,0.031161


In [68]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [69]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [70]:
to_publish.to_csv('1_12_2.csv', index=False)